**Preface:** 
* Inferencing R1 is hard as empty responses are quite frequent

In [ ]:
import pandas as pd

gsm_data = pd.read_csv('../data/preprocessed/gsm_data.csv')
gsm_data = gsm_data.sample(frac = 1)

**DONT RUN THE ABOVE AFTER RUNNING ONCE if running the below code more than 1 time** \
It ensures all types of questions have a fair chance of being answered by the LLM \
GSM-symbolic groups questions together based on template. This causes bias for the first few templates.  

In [ ]:
import requests
import json

def ask_r1(prompt):
  response = requests.post(
    url="https://openrouter.ai/api/v1/chat/completions",
    headers={
      "Authorization": "Bearer 'api-key' ",
      "Content-Type": "application/json",
    },
    data=json.dumps({
      "model": "deepseek/deepseek-r1",
      "messages": [
        {
          "role": "user",
          "content": prompt
        }
      ]})
  )

  response_json = json.loads(response.content)
  response_json = response_json.get("choices", [{}])[0].get("message", {}).get("content", "No content found")
  return response_json

In [5]:
import re

def extract_final_ans(response):
    answer = response.replace('*', '')
    answer = answer.replace("," ,'')
    answer = answer.lower().split('answer: ')

    try:
        final = re.findall(r"\d+", answer[-1])[0]
    except IndexError:
        final = None
    
    return final

### Collect 50 incorrect solutions first:

In [ ]:
import time 

def collect_responses(df, mistakes):
    llm_response = []
    n = 0

    for idx, row in df.iterrows(): # once number of desired mistakes is reached, exit function
        if n==mistakes:
            return llm_response
        
        question = row['question']
        answer = row['answer']
        
        prompt = f"""
        Answer the following question. Please use Chain of Thought and number each step:
        {question}
        
        Have your final answer strictly at the end of your response in this form:
        Answer: 
        """
        
        response = ask_r1(prompt)
        final = extract_final_ans(response)
    
        while final == None:
            print('Response was None')
            time.sleep(15)
            
            response = ask_r1(prompt)
            final = extract_final_ans(response)
        
        ref_final = answer.splitlines()
        ref_final = ref_final[-1].replace("#",'')
        ref_final = ref_final.strip()

        if ref_final != final:
            n += 1
            llm_response.append({'question': question, 'ref_answer': answer, 'llm_answer': response, 'final_ans': final})
            temp = pd.DataFrame(llm_response)
            temp.to_csv('../data/responses/temp.csv')
            
        print("final_ans: ", final)
        print("ref_ans: ", ref_final)
        print('iteration: ', n)

    return llm_response

In [ ]:
llm_gsm = collect_responses(gsm_data, 100) # gather more than 50 incorrct solutions to be able to ensure you have 50 valid samples after filtering
llm_gsm

In [ ]:
save_directory = "../data/responses"
df_gsm= pd.DataFrame(llm_gsm)
df_gsm.to_csv(save_directory + '/r1_responses.csv')

## Then Self correct with the following:

### Self correction process - w/ error location and error type

In [ ]:
import pandas as pd

df_responses = pd.read_csv('../data/responses/r1_response_annotated.csv')
df_responses = df_responses.head(50)

In [ ]:
import re

def self_correct(df):
    llm_response_corrected = []

    for idx, row in df.iterrows():
        question = row['question']
        solution = row['llm_answer']
        error_step = row['Error Step']
        error_type = re.sub(r'^[\d.]+', '', row['Error Reason'].strip('()')).lstrip()
        answer = row['ref_answer']

        correction_prompt = f"""
        Here are definitions of error types:
        - Incorrect value of variable cited or used: If the numerical value of a known (input or intermediate) variable is cited or used incorrectly.
        - Value of a known variable calculated again: If the value of an input or intermediate (determined mid solution) variable is calculated when it's value is already provided/ known.
        - Irrelevant/incorrect variable cited or used: If a variable in a solution is found to lie outside of the ideal variable-set of the solutioin (thereby making the solution innacurate).
        - Relevant variable missing: If any of the required variables in a solutioin for a target variable is missing (thereby making the solution innacurate).
        - Unit Inconsistency: If during the execution of an expression, two or more unit incompatible quantities are operated between.
        - Calculation Error: If the resultant numerical value of an operation or set of operations between numerical values is incorrect. 
        - Incorrect Value: If the numerical value of the final answer is incorrect.
        
        The following solution of the given question is generated using the same model as you. Consider the question and solution that uses the Chain of Thought strategy with each step numbered.

        Question: {question}

        Solution: {solution}

        You made a mistake in Step {int(error_step)} due to {error_type}.  Can you revise your solution and carefully solve it again?
        """

        response = ask_r1(correction_prompt)
        final = extract_final_ans(response)

        ref_final = answer.splitlines()
        ref_final = ref_final[-1].replace("#",'')
        ref_final = ref_final.strip()

        # Monitoring run
        print("Ref answer: ", ref_final)
        print("LLM initial answer: ", row['final_ans'])
        print("LLM self revised: ", final)
        
        llm_response_corrected.append({'question': question, 'ref_answer': answer, 'llm_initial_answer': row['llm_answer'],
                                        'initial_final_ans': row['final_ans'], 'llm_revised_answer': response,
                                        'revised_final_ans': final, 'error_type': row['Error Reason'], 
                                        'error_step': row['Error Step'], 'total_steps': row['Total Steps'],
                                        'correct': ref_final==final})
    
    return llm_response_corrected
        

In [6]:
llm_self_corrected = self_correct(df_responses)

Ref answer:  16
LLM initial answer:  8
LLM self revised:  8
Ref answer:  44
LLM initial answer:  26
LLM self revised:  None
Ref answer:  220
LLM initial answer:  1260
LLM self revised:  220
Ref answer:  757
LLM initial answer:  797
LLM self revised:  797
Ref answer:  1138
LLM initial answer:  1198
LLM self revised:  1198
Ref answer:  3
LLM initial answer:  0
LLM self revised:  0
Ref answer:  1014
LLM initial answer:  1106
LLM self revised:  1014
Ref answer:  2713
LLM initial answer:  25
LLM self revised:  2797
Ref answer:  1908
LLM initial answer:  1900
LLM self revised:  1908
Ref answer:  110
LLM initial answer:  440
LLM self revised:  110
Ref answer:  14
LLM initial answer:  0
LLM self revised:  0
Ref answer:  317
LLM initial answer:  342
LLM self revised:  317
Ref answer:  84
LLM initial answer:  180
LLM self revised:  84
Ref answer:  62
LLM initial answer:  136
LLM self revised:  136
Ref answer:  14
LLM initial answer:  0
LLM self revised:  0
Ref answer:  120
LLM initial answer:  1

In [ ]:
save_directory = "../data/responses"

df_corrected = pd.DataFrame(llm_self_corrected)
df_corrected.to_csv(save_directory + '/r1_self_corrected.csv')

### Self correction process - general error location w/ error type

In [ ]:
import re

def self_correct(df):
    llm_response_corrected = []

    for idx, row in df.iterrows():
        question = row['question']
        solution = row['llm_answer']
        error_type = re.sub(r'^[\d.]+', '', row['Error Reason'].strip('()')).lstrip()
        answer = row['ref_answer']


        if row['Error Step'] <= (row['Total Steps']/2):
            prompt = f"""
            Here are definitions of error types:
            - Incorrect value of variable cited or used: If the numerical value of a known (input or intermediate) variable is cited or used incorrectly.
            - Value of a known variable calculated again: If the value of an input or intermediate (determined mid solution) variable is calculated when it's value is already provided/ known.
            - Irrelevant/incorrect variable cited or used: If a variable in a solution is found to lie outside of the ideal variable-set of the solutioin (thereby making the solution innacurate).
            - Relevant variable missing: If any of the required variables in a solutioin for a target variable is missing (thereby making the solution innacurate).
            - Unit Inconsistency: If during the execution of an expression, two or more unit incompatible quantities are operated between.
            - Calculation Error: If the resultant numerical value of an operation or set of operations between numerical values is incorrect. 
            - Incorrect Value: If the numerical value of the final answer is incorrect.

            The following solution is generated using the same model as you. Consider the following question and solution that uses the Chain of Thought strategy with each step numbered.

            Question: {question}

            Solution: {solution}

            You made a mistake in the upper half of your solution due to {error_type}.  Can you revise your solution and carefully solve it again?
            """
        else:
            prompt = f"""
            Here are definitions of error types:
            - Incorrect value of variable cited or used: If the numerical value of a known (input or intermediate) variable is cited or used incorrectly.
            - Value of a known variable calculated again: If the value of an input or intermediate (determined mid solution) variable is calculated when it's value is already provided/ known.
            - Irrelevant/incorrect variable cited or used: If a variable in a solution is found to lie outside of the ideal variable-set of the solutioin (thereby making the solution innacurate).
            - Relevant variable missing: If any of the required variables in a solutioin for a target variable is missing (thereby making the solution innacurate).
            - Unit Inconsistency: If during the execution of an expression, two or more unit incompatible quantities are operated between.
            - Calculation Error: If the resultant numerical value of an operation or set of operations between numerical values is incorrect. 
            - Incorrect Value: If the numerical value of the final answer is incorrect.
            
            The following solution is generated using the same model as you. Consider the following question and solution that uses the Chain of Thought strategy with each step numbered.

            Question: {question}

            Solution: {solution}

            You made a mistake in the latter half of your solution due to {error_type}.  Can you revise your solution and carefully solve it again?
            """

        response = ask_r1(prompt)
        final = extract_final_ans(response)

        ref_final = answer.splitlines()
        ref_final = ref_final[-1].replace("#",'')
        ref_final = ref_final.strip()

        print("Ref answer: ", ref_final)
        print("LLM initial answer: ", row['final_ans'])
        print("LLM self revised: ", final)
        
        llm_response_corrected.append({'question': question, 'ref_answer': answer, 'llm_initial_answer': row['llm_answer'],
                                        'initial_final_ans': row['final_ans'], 'llm_revised_answer': response,
                                        'revised_final_ans': final, 'error_type': row['Error Reason'], 
                                        'error_step': row['Error Step'], 'total_steps': row['Total Steps'],
                                        'correct': ref_final==final})
        temp = pd.DataFrame(llm_response_corrected)
        temp.to_csv('temp.csv')
    
    return llm_response_corrected
        

In [9]:
llm_self_corrected = self_correct(df_responses)

Ref answer:  16
LLM initial answer:  8
LLM self revised:  8
Ref answer:  44
LLM initial answer:  26
LLM self revised:  26
Ref answer:  220
LLM initial answer:  1260
LLM self revised:  1260
Ref answer:  757
LLM initial answer:  797
LLM self revised:  797
Ref answer:  1138
LLM initial answer:  1198
LLM self revised:  1198
Ref answer:  3
LLM initial answer:  0
LLM self revised:  3
Ref answer:  1014
LLM initial answer:  1106
LLM self revised:  1014
Ref answer:  2713
LLM initial answer:  25
LLM self revised:  25
Ref answer:  1908
LLM initial answer:  1900
LLM self revised:  1
Ref answer:  110
LLM initial answer:  440
LLM self revised:  110
Ref answer:  14
LLM initial answer:  0
LLM self revised:  0
Ref answer:  317
LLM initial answer:  342
LLM self revised:  317
Ref answer:  84
LLM initial answer:  180
LLM self revised:  84
Ref answer:  62
LLM initial answer:  136
LLM self revised:  136
Ref answer:  14
LLM initial answer:  0
LLM self revised:  0
Ref answer:  120
LLM initial answer:  168
LLM

In [ ]:
save_directory = "../data/responses"

df_corrected = pd.DataFrame(llm_self_corrected)
df_corrected.to_csv(save_directory + '/r1_selfcorrected_genloc.csv')

### Self correction process - with no error location

In [ ]:
import re

def self_correct(df):
    llm_response_corrected = []

    for idx, row in df.iterrows():
        question = row['question']
        solution = row['llm_answer']
        error_type = re.sub(r'^[\d.]+', '', row['Error Reason'].strip('()')).lstrip()
        answer = row['ref_answer']

        prompt = f"""
        Here are definitions of error types:
        - Incorrect value of variable cited or used: If the numerical value of a known (input or intermediate) variable is cited or used incorrectly.
        - Value of a known variable calculated again: If the value of an input or intermediate (determined mid solution) variable is calculated when it's value is already provided/ known.
        - Irrelevant/incorrect variable cited or used: If a variable in a solution is found to lie outside of the ideal variable-set of the solutioin (thereby making the solution innacurate).
        - Relevant variable missing: If any of the required variables in a solutioin for a target variable is missing (thereby making the solution innacurate).
        - Unit Inconsistency: If during the execution of an expression, two or more unit incompatible quantities are operated between.
        - Calculation Error: If the resultant numerical value of an operation or set of operations between numerical values is incorrect. 
        - Incorrect Value: If the numerical value of the final answer is incorrect.
        
        The following solution is generated using the same model as you. Consider the following question and solution that uses the Chain of Thought strategy with each step numbered.

        Question: {question}

        Solution: {solution}

        You made a mistake in your solution due to {error_type}.  Can you revise your solution and carefully solve it again?
        """

        response = ask_r1(prompt)
        final = extract_final_ans(response)

        ref_final = answer.splitlines()
        ref_final = ref_final[-1].replace("#",'')
        ref_final = ref_final.strip()

        print("Ref answer: ", ref_final)
        print("LLM initial answer: ", row['final_ans'])
        print("LLM self revised: ", final)
        
        llm_response_corrected.append({'question': question, 'ref_answer': answer, 'llm_initial_answer': row['llm_answer'],
                                        'initial_final_ans': row['final_ans'], 'llm_revised_answer': response,
                                        'revised_final_ans': final, 'error_type': row['Error Reason'], 
                                        'error_step': row['Error Step'], 'total_steps': row['Total Steps'],
                                        'correct': ref_final==final})
    
    return llm_response_corrected
        

In [14]:
llm_self_corrected = self_correct(df_responses)

Ref answer:  16
LLM initial answer:  8
LLM self revised:  8
Ref answer:  44
LLM initial answer:  26
LLM self revised:  26
Ref answer:  220
LLM initial answer:  1260
LLM self revised:  220
Ref answer:  757
LLM initial answer:  797
LLM self revised:  797
Ref answer:  1138
LLM initial answer:  1198
LLM self revised:  10
Ref answer:  3
LLM initial answer:  0
LLM self revised:  None
Ref answer:  1014
LLM initial answer:  1106
LLM self revised:  1036
Ref answer:  2713
LLM initial answer:  25
LLM self revised:  25
Ref answer:  1908
LLM initial answer:  1900
LLM self revised:  1908
Ref answer:  110
LLM initial answer:  440
LLM self revised:  110
Ref answer:  14
LLM initial answer:  0
LLM self revised:  0
Ref answer:  317
LLM initial answer:  342
LLM self revised:  289
Ref answer:  84
LLM initial answer:  180
LLM self revised:  84
Ref answer:  62
LLM initial answer:  136
LLM self revised:  136
Ref answer:  14
LLM initial answer:  0
LLM self revised:  0
Ref answer:  120
LLM initial answer:  168


In [ ]:
save_directory = "../data/responses"

df_corrected = pd.DataFrame(llm_self_corrected)
df_corrected.to_csv(save_directory + '/r1_selfcorrected_noloc.csv')

### Baseline: No error location or error type

In [ ]:
import re

def self_correct(df):
    llm_response_corrected = []

    for idx, row in df.iterrows():
        question = row['question']
        solution = row['llm_answer']
        answer = row['ref_answer']

        prompt = f"""
        The following solution is generated using the same model as you. Consider the following question and solution that uses the Chain of Thought strategy with each step numbered.

        Question: {question}

        Solution: {solution}

        You made a mistake in your solution.  Can you revise your solution and carefully solve it again?
        """

        response = ask_r1(prompt)
        final = extract_final_ans(response)

        ref_final = answer.splitlines()
        ref_final = ref_final[-1].replace("#",'')
        ref_final = ref_final.strip()

        print("Ref answer: ", ref_final)
        print("LLM initial answer: ", row['final_ans'])
        print("LLM self revised: ", final)
        
        llm_response_corrected.append({'question': question, 'ref_answer': answer, 'llm_initial_answer': row['llm_answer'],
                                        'initial_final_ans': row['final_ans'], 'llm_revised_answer': response,
                                        'revised_final_ans': final, 'error_type': row['Error Reason'], 
                                        'error_step': row['Error Step'], 'total_steps': row['Total Steps'],
                                        'correct': ref_final==final})
    
    return llm_response_corrected
        

In [ ]:
llm_self_corrected_1 = self_correct(df_responses)

Ref answer:  16
LLM initial answer:  8
LLM self revised:  8
Ref answer:  44
LLM initial answer:  26
LLM self revised:  1
Ref answer:  220
LLM initial answer:  1260
LLM self revised:  220
Ref answer:  757
LLM initial answer:  797
LLM self revised:  797
Ref answer:  1138
LLM initial answer:  1198
LLM self revised:  1198
Ref answer:  3
LLM initial answer:  0
LLM self revised:  0
Ref answer:  1014
LLM initial answer:  1106
LLM self revised:  1014
Ref answer:  2713
LLM initial answer:  25
LLM self revised:  25
Ref answer:  1908
LLM initial answer:  1900
LLM self revised:  1908
Ref answer:  110
LLM initial answer:  440
LLM self revised:  110
Ref answer:  14
LLM initial answer:  0
LLM self revised:  0
Ref answer:  317
LLM initial answer:  342
LLM self revised:  317
Ref answer:  84
LLM initial answer:  180
LLM self revised:  84
Ref answer:  62
LLM initial answer:  136
LLM self revised:  120
Ref answer:  14
LLM initial answer:  0
LLM self revised:  0
Ref answer:  120
LLM initial answer:  168
LL

In [ ]:
save_directory = "../data/responses"

df_corrected = pd.DataFrame(llm_self_corrected_1)
df_corrected.to_csv(save_directory + '/r1_selfcorrected_baseline.csv')